In [1]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

2023-06-05 14:51:35.957102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Set the path to your dataset
train_dir = 'data/train'
test_dir = 'data/test'
val_dir = 'data/val'

# Set the input image dimensions
input_shape = (256, 256, 1)

# Set the number of classes
num_classes = 6

# Create an ImageDataGenerator for normalization and grayscale conversion
data_generator = ImageDataGenerator(rescale=1.0/255.0)

# Create data iterators for training, validation, and testing
train_iterator = data_generator.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32
)

val_iterator = data_generator.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32
)

test_iterator = data_generator.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=1,
    shuffle=False
)


Found 6942 images belonging to 6 classes.
Found 360 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [3]:
# Define the model architecture
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=16, max_value=64, step=16),
                            (3, 3),
                            activation='relu',
                            input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32),
                            (3, 3),
                            activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=64, max_value=256, step=64),
                            (3, 3),
                            activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64),
                           activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [4]:
# Instantiate the Keras Tuner RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='tf_tuner_directory',
    project_name='tf_tuner') 

# Perform the hyperparameter search
# tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))
tuner.search(train_iterator, epochs=10, validation_data=val_iterator)

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 10 Complete [00h 06m 41s]
val_accuracy: 0.9444444179534912

Best val_accuracy So Far: 0.949999988079071
Total elapsed time: 01h 00m 06s
INFO:tensorflow:Oracle triggered exit


In [3]:
# Define the model architecture
def build_model2(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=16, max_value=64, step=16),
                            (3, 3),
                            activation='relu',
                            input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32),
                            (3, 3),
                            activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=64, max_value=256, step=64),
                            (3, 3),
                            activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64),
                           activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1)))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate',
                                                                         values=[1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the Keras Tuner RandomSearch tuner
tuner3 = RandomSearch(
    build_model2,
    objective='val_loss',
    max_trials=50,
    directory='tf_tuner_directory',
    project_name='tf_tuner3') 

# Perform the hyperparameter search
# tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))
tuner3.search(train_iterator, epochs=10, validation_data=val_iterator)

# Get the best model and hyperparameters
best_model = tuner3.get_best_models(num_models=1)[0]
best_hyperparameters = tuner3.get_best_hyperparameters(num_trials=1)[0]

Trial 50 Complete [00h 06m 47s]
val_loss: 0.18060743808746338

Best val_loss So Far: 0.17245377600193024
Total elapsed time: 03h 52m 17s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model

In [ ]:
best_hyperparameters

In [ ]:
best